In [ ]:
# Details:
# Study: Brain Network Organization and Behavior
# Note: This notebook extracts the adjacency matrix all subjects using the Seitzman et al. (2020) atlas
# Note: All paths are relative to the current working directory
# Note: Subjects should be organized into directories that encode the given experimental condition

# Credits:
# Written by Richard Huskey
# https://nilearn.github.io/auto_examples/03_connectivity/plot_group_level_connectivity.html#sphx-glr-auto-examples-03-connectivity-plot-group-level-connectivity-py


# Notes and dependencies
# None

In [ ]:
import numpy as np
import glob
from nilearn import datasets
import os
os.system('mkdir -p time_series_01608') 

In [ ]:
# make meta-data to distinguish flow,bore,and frus
import pandas
colnames = ['sub', 'condition', 'file']
metadata_scan_full = pandas.read_csv('/data/meta_full.csv', names=colnames)
sub_full = metadata_scan_full['sub'].drop([0],axis=0).tolist()
condition_full = metadata_scan_full['condition'].drop([0],axis=0).tolist()
file_full = metadata_scan_full['file'].drop([0],axis=0).tolist()
file_full = [s.replace('.nii.gz', '') for s in file_full]
print(file_full)

In [ ]:
filenames_full = []
for i in file_full:
    filenames_full.append('/data/merger_01608_Seitzman/{}.nii.gz'.format(i))

In [ ]:
filenames_full

In [ ]:
from nilearn import input_data

In [ ]:
# Load the coordinate of the Seitzman et al. (2020) atlas
new_atlas = pandas.read_table("/data/analysis/ROIs_300inVol_MNI_allInfo.txt", 
                              delim_whitespace=True)

index_5mm = new_atlas['radius(mm)'] == 5
index_4mm = new_atlas['radius(mm)'] == 4

In [ ]:
# Makes a table with the x,y,z coordinates of each seed ROI

coords = np.vstack((new_atlas['x'], new_atlas['y'], new_atlas['z'])).T
coords_5mm = coords[index_5mm,:]
coords_4mm = coords[index_4mm,:]
print('Stacked power coordinates in array of shape {0}.'.format(coords_4mm.shape))

In [ ]:
# Now, lets make some masks
# Note, these are 5mm spheres for cortical regions and 4mm spheres for subcortical regions

masker_5mm = input_data.NiftiSpheresMasker(
    seeds=coords_5mm, smoothing_fwhm=None, radius=5.,
    detrend=True, t_r=2.0, memory='nilearn_cache', memory_level=1)
masker_4mm = input_data.NiftiSpheresMasker(
    seeds=coords_4mm, smoothing_fwhm=None, radius=4,
    detrend=True, t_r=2.0, allow_overlap = True, memory='nilearn_cache', memory_level=1)

In [ ]:
# We transform the fMRI images to time series for each node by averaging voxels in the defined sphere nodes
# Then we append the time series to the list
pooled_subjects = []

for f in filenames_full:
    print("extracting {} ".format(f))
    time_series_5mm = masker_5mm.fit_transform(f)
    time_series_4mm = masker_4mm.fit_transform(f)
    time_series = np.hstack((time_series_5mm,time_series_4mm))
    
    pooled_subjects.append(time_series)
    print("just finished {} ".format(f))
    

In [ ]:
len(pooled_subjects)

In [ ]:
# Make sure that each time series is a 120 (TR) * 300 (nodes) matrix
for i in range(len(pooled_subjects)):
    assert pooled_subjects[i].shape[0] == 120
    assert pooled_subjects[i].shape[1] == 300

In [ ]:
# save the time series data as txt files
out_file_names=[s.replace('_bold_space-MNI152NLin2009aAsym_preproc_merged', '') for s in file_full]
out_file_names=[s.replace('_task-game', '') for s in out_file_names]

for i in range(len(pooled_subjects)):
    np.savetxt('time_series_01608/{}.out'.format(out_file_names[i]), pooled_subjects[i], delimiter=',')
    
